In [2]:
import csv
import numpy as np
import pandas as pd
import sys, os
import re
import ast
import datetime as dt
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
diag = pd.read_csv('../data/hos/diagnoses_icd.csv').reset_index(drop = True)
# diag = diag.reset_index(inplace=True)
diag

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4756321,19999987,23865745,7,41401,9
4756322,19999987,23865745,8,78039,9
4756323,19999987,23865745,9,0413,9
4756324,19999987,23865745,10,36846,9


In [4]:
diag['icd_version'].value_counts()
# diag.shape #(4756326, 6)

9     2766877
10    1989449
Name: icd_version, dtype: int64

In [5]:
diag['new'] = diag['icd_code']

In [6]:
#adding underscore with the ICD version on the ICD_codes
diag.loc[diag["icd_version"] == 9, "new"] = '9_' + diag.loc[diag["icd_version"] == 9, "new"]
diag.loc[diag["icd_version"] == 10, "new"] = '10_' + diag.loc[diag["icd_version"] == 10, "new"]


In [7]:
diag.shape
diag.loc[diag["icd_version"] == 10]

,subject_id,hadm_id,seq_num,icd_code,icd_version,new
40,10000084,23052089,1,G3183,10,10_G3183
41,10000084,23052089,2,F0280,10,10_F0280
42,10000084,23052089,3,R441,10,10_R441
43,10000084,23052089,4,R296,10,10_R296
44,10000084,23052089,5,E785,10,10_E785
...,...,...,...,...,...,...
4756291,19999828,29734428,18,Z9049,10,10_Z9049
4756292,19999828,29734428,19,Z87891,10,10_Z87891
4756293,19999828,29734428,20,B9620,10,10_B9620
4756294,19999828,29734428,21,Z1611,10,10_Z1611


In [8]:
d_diag = pd.read_csv('../data/hos/d_icd_diagnoses.csv').reset_index(drop = True)


# diag = diag.reset_index(inplace=True)
d_diag #109775 rows × 3 columns

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [9]:
d_diag.loc[d_diag['icd_version']==9]['icd_code'].nunique()#there are 14666 unique ICD code in version 9
d_diag.loc[d_diag['icd_version']==10]['icd_code'].nunique() #there are 95109 unique ICD code in version 10
diag.loc[diag['icd_version']==9]['icd_code'].nunique()#there are 9072 unique ICD code in version 9
diag.loc[diag['icd_version']==10]['icd_code'].nunique() #there are 16757 unique ICD code in version 10

16757

In [10]:
(diag
 .merge(d_diag, how='inner', left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version'])
.reset_index())

,index,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title
0,0,10000032,22595853,1,5723,9,9_5723,Portal hypertension
1,1,10000826,20032235,4,5723,9,9_5723,Portal hypertension
2,2,10000826,28289260,1,5723,9,9_5723,Portal hypertension
3,3,10005866,26158160,4,5723,9,9_5723,Portal hypertension
4,4,10008924,23676183,7,5723,9,9_5723,Portal hypertension
...,...,...,...,...,...,...,...,...
4756321,4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini..."
4756322,4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ..."
4756323,4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,..."
4756324,4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...


In [11]:
diag_d = (diag
          .merge(d_diag, how='inner', left_on=['icd_code', 'icd_version'], 
        right_on=['icd_code', 'icd_version']))

In [12]:
print(diag_d.shape)
diag_d.head()

(4756326, 7)


,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension


In [13]:
d_diag

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [14]:
mappingicd9_icd10 = pd.read_csv("ICD9_to_ICD10_mapping.txt", delimiter='\t')

In [15]:
mappingicd9_icd10

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
0,ICD9,996.76,OTHER COMPLICATIONS DUE TO GENITOURINARY DEVIC...,99676,T8384XA,10000
1,ICD9,V54.12,AFTERCARE FOR HEALING TRAUMATIC FRACTURE OF LO...,V5412,S52602D,10000
2,ICD9,730.06,ACUTE OSTEOMYELITIS INVOLVING LOWER LEG,73006,M86169,10000
3,ICD9,345.61,"INFANTILE SPASMS, WITH INTRACTABLE EPILEPSY",34561,G40824,10000
4,ICD9,989.5,TOXIC EFFECT OF VENOM,9895,T63421A,10000
...,...,...,...,...,...,...
24280,ICD9,819,"MULTIPLE FRACTURES INVOLVING BOTH UPPER LIMBS,...",8190,S5290XA,10121
24281,ICD9,E888.0,FALL RESULTING IN STRIKING AGAINST SHARP OBJECT,E8880,W01110A,10000
24282,ICD9,V48.4,SENSORY PROBLEM WITH HEAD,V484,R6889,10000
24283,ICD9,802.25,ANGLE OF JAW CLOSED FRACTURE,80225,S02650A,10000


In [16]:
mappingicd9_icd10["icd9cm"].nunique() #12538

12538

In [17]:
mappingicd9_icd10.columns
mappingicd9_icd10.loc[mappingicd9_icd10['icd9cm']== '5723']

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
5589,ICD9,572.3,PORTAL HYPERTENSION,5723,K766,0


In [18]:
diag_d.loc[diag_d['icd_version']==9].shape #(2766877, 8) 5726301 rows × 10 columns
diag_d.loc[diag_d['icd_version']==10].shape #(1989449, 8)
diag_d['icd_version'].isin([9, 10]).shape #4756326

(4756326,)

In [19]:
2766877 - 5726301
# 4756326-5726301

-2959424

In [20]:
df9 = diag_d.loc[diag_d['icd_version']==9]
df9['icd_code'].value_counts()

4019     97361
2724     63875
53081    46238
25000    41029
42731    35180
         ...  
36119        1
37245        1
24971        1
3020         1
E8736        1
Name: icd_code, Length: 9072, dtype: int64

In [21]:
df9.shape #(2766877, 8))
diag_d['icd_version'].value_counts()

9     2766877
10    1989449
Name: icd_version, dtype: int64

In [22]:
diag_d['icd_version'].value_counts()

5717150-2766877 #2950273 mapping 24285row

2950273

In [23]:
diag_d

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension
...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini..."
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ..."
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,..."
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...


In [24]:
mappingicd9_icd10.drop_duplicates()

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
0,ICD9,996.76,OTHER COMPLICATIONS DUE TO GENITOURINARY DEVIC...,99676,T8384XA,10000
1,ICD9,V54.12,AFTERCARE FOR HEALING TRAUMATIC FRACTURE OF LO...,V5412,S52602D,10000
2,ICD9,730.06,ACUTE OSTEOMYELITIS INVOLVING LOWER LEG,73006,M86169,10000
3,ICD9,345.61,"INFANTILE SPASMS, WITH INTRACTABLE EPILEPSY",34561,G40824,10000
4,ICD9,989.5,TOXIC EFFECT OF VENOM,9895,T63421A,10000
...,...,...,...,...,...,...
24280,ICD9,819,"MULTIPLE FRACTURES INVOLVING BOTH UPPER LIMBS,...",8190,S5290XA,10121
24281,ICD9,E888.0,FALL RESULTING IN STRIKING AGAINST SHARP OBJECT,E8880,W01110A,10000
24282,ICD9,V48.4,SENSORY PROBLEM WITH HEAD,V484,R6889,10000
24283,ICD9,802.25,ANGLE OF JAW CLOSED FRACTURE,80225,S02650A,10000


In [25]:
mappingicd9_icd10['icd9cm'].value_counts()

V5412    533
V5416    528
V5411    213
V5413    159
99529    159
        ... 
73016      1
65973      1
76079      1
05472      1
V484       1
Name: icd9cm, Length: 12538, dtype: int64

In [26]:
mappingicd9_icd10.loc[mappingicd9_icd10['icd9cm']=='V5412'].drop_duplicates('icd9cm')

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
1,ICD9,V54.12,AFTERCARE FOR HEALING TRAUMATIC FRACTURE OF LO...,V5412,S52602D,10000


In [27]:
(diag_d.merge
 (mappingicd9_icd10[['icd9cm', 'icd10cm']], 
  how='inner', left_on=['icd_code'], right_on=['icd9cm']))

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766
...,...,...,...,...,...,...,...,...,...
5726296,19975121,28839110,3,0538,9,9_0538,Herpes zoster with unspecified complication,0538,B028
5726297,19983257,21588174,17,E9463,9,9_E9463,"Emollients, demulcents, and protectants causin...",E9463,NoDx
5726298,19985683,25819866,6,2109,9,9_2109,"Benign neoplasm of pharynx, unspecified",2109,D109
5726299,19989305,29998115,2,74682,9,9_74682,Cor triatriatum,74682,Q242


In [28]:
(df9.merge(mappingicd9_icd10[['icd9cm', 'icd10cm']], 
           how='inner', left_on=['icd_code'], right_on=['icd9cm']))

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766
...,...,...,...,...,...,...,...,...,...
5717145,19975121,28839110,3,0538,9,9_0538,Herpes zoster with unspecified complication,0538,B028
5717146,19983257,21588174,17,E9463,9,9_E9463,"Emollients, demulcents, and protectants causin...",E9463,NoDx
5717147,19985683,25819866,6,2109,9,9_2109,"Benign neoplasm of pharynx, unspecified",2109,D109
5717148,19989305,29998115,2,74682,9,9_74682,Cor triatriatum,74682,Q242


In [29]:
(diag_d.merge
 (mappingicd9_icd10[['icd9cm', 'icd10cm']], 
  how='inner', left_on=['icd_code'], right_on=['icd9cm'])).loc[(diag_d.merge
 (mappingicd9_icd10[['icd9cm', 'icd10cm']], 
  how='inner', left_on=['icd_code'], right_on=['icd9cm']))['icd_code']=='V5412']

#outer =7751133 rows

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
5042519,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S52602D
5042520,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S52389D
5042521,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S52559D
5042522,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S52366D
5042523,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S52355D
...,...,...,...,...,...,...,...,...,...
5060103,19523301,22310739,5,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S52614D
5060104,19523301,22310739,5,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S52212D
5060105,19523301,22310739,5,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S59041D
5060106,19523301,22310739,5,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V5412,S52221D


In [30]:
# diag_d.loc[diag_d['icd_3']=='272']

In [31]:
regex = r"[H/h]eart"

# regex = r"[K/k]idney"

regex = r"[D/d]igestive"

# test_str = diag_d['long_title']

# matches = re.findall(regex, test_str, re.MULTILINE)


# diag_d['long_title'].str.findall('[p/P]ortal')

diag_d[diag_d.long_title.str.match(regex)] #matcehs the beigning of the line

diag_d[diag_d.long_title.str.contains(regex)] #matches the pattern anywhere in the text

diag_d[diag_d.long_title.str.contains(regex)]['icd_code'].value_counts()

Z9049     2134
Z800      1506
99749     1311
9974      1293
K9189      674
          ... 
Q459         1
A1883        1
Z871         1
O99619       1
V127         1
Name: icd_code, Length: 64, dtype: int64

In [32]:
diag_d["icd_3"]=diag_d['icd_code'].astype(str).str[:3]
diag_d

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572
...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",T24
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",O30
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",H35
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,K80


In [33]:
diag_d['icd_3'].value_counts().to_frame().reset_index().rename(columns={'index':'icd_code3',
                                                                       'icd_3': 'counts'})

,icd_code3,counts
0,401,101252
1,272,83708
2,250,75310
3,V58,69592
4,E87,66157
...,...,...
2585,763,1
2586,N07,1
2587,V39,1
2588,N33,1


In [34]:
print(mappingicd9_icd10.shape)
mappingicd9_icd10.head()

(24285, 6)


,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
0,ICD9,996.76,OTHER COMPLICATIONS DUE TO GENITOURINARY DEVIC...,99676,T8384XA,10000
1,ICD9,V54.12,AFTERCARE FOR HEALING TRAUMATIC FRACTURE OF LO...,V5412,S52602D,10000
2,ICD9,730.06,ACUTE OSTEOMYELITIS INVOLVING LOWER LEG,73006,M86169,10000
3,ICD9,345.61,"INFANTILE SPASMS, WITH INTRACTABLE EPILEPSY",34561,G40824,10000
4,ICD9,989.5,TOXIC EFFECT OF VENOM,9895,T63421A,10000


In [35]:
diag.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,new
0,10000032,22595853,1,5723,9,9_5723
1,10000032,22595853,2,78959,9,9_78959
2,10000032,22595853,3,5715,9,9_5715
3,10000032,22595853,4,07070,9,9_07070
4,10000032,22595853,5,496,9,9_496


In [36]:
d_diag.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [37]:
diag_d.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572


In [38]:
icd9_icd10_dropduplicates = mappingicd9_icd10.drop_duplicates('icd9cm')

In [39]:
# df.drop(['B', 'C'], axis=1)
#diag_d.drop(columns=['icd_3']) #4756326 rows × 7 columns


((diag_d.drop(columns=['icd_3']))
 .merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], how = 'inner',
       left_on= ['icd_code'],
       right_on=['icd9cm']).loc[(diag_d.drop(columns=['icd_3']))
 .merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], how = 'inner',
       left_on= ['icd_code'],
       right_on=['icd9cm'])['icd_version']==10])


# (df9.merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], 
#            how = 'inner',
#            left_on= ['icd_code'],
#            right_on=['icd9cm']))  #2727971 rows
# (df9.merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], 
#            how = 'outer',
#            left_on= ['icd_code'],
#            right_on=['icd9cm'])) #2770706


,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
1195274,10001401,24818636,12,E8342,10,10_E8342,Hypomagnesemia,E8342,V9339XA
1195275,10005606,22751751,8,E8342,10,10_E8342,Hypomagnesemia,E8342,V9339XA
1195276,10010663,22209635,10,E8342,10,10_E8342,Hypomagnesemia,E8342,V9339XA
1195277,10012206,23961896,15,E8342,10,10_E8342,Hypomagnesemia,E8342,V9339XA
1195278,10023328,20884955,6,E8342,10,10_E8342,Hypomagnesemia,E8342,V9339XA
...,...,...,...,...,...,...,...,...,...
2732062,18395069,24505224,5,E8301,10,10_E8301,Wilson's disease,E8301,V9089XA
2732063,18821656,21824819,6,E8301,10,10_E8301,Wilson's disease,E8301,V9089XA
2735240,14480975,27414700,14,E8330,10,10_E8330,"Disorder of phosphorus metabolism, unspecified",E8330,V9338XA
2735241,19400460,27338264,2,E8330,10,10_E8330,"Disorder of phosphorus metabolism, unspecified",E8330,V9338XA


In [40]:
2727971 - 2766877 #38906

2770706 - 2766877 #3829

2737057 - 4756326 #-2019269

-2019269

In [42]:
#merging the diag_d with the mapping ICD9 and icd10 files

print(diag_d.shape)
print(icd9_icd10_dropduplicates.shape)

# df['col1'] = df['col1'].fillna(df['col2'])

diag_d_icd = ((diag_d.drop(columns=['icd_3']))
 .merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], how = 'left',
       left_on= ['icd_code'],
       right_on=['icd9cm']))

print(diag_d_icd.shape)
print(diag_d_icd.head())
print(diag_d_icd.tail())

(4756326, 8)
(12538, 6)
(4756326, 9)
   subject_id   hadm_id  seq_num icd_code  icd_version     new  \
0    10000032  22595853        1     5723            9  9_5723   
1    10000826  20032235        4     5723            9  9_5723   
2    10000826  28289260        1     5723            9  9_5723   
3    10005866  26158160        4     5723            9  9_5723   
4    10008924  23676183        7     5723            9  9_5723   

            long_title icd9cm icd10cm  
0  Portal hypertension   5723    K766  
1  Portal hypertension   5723    K766  
2  Portal hypertension   5723    K766  
3  Portal hypertension   5723    K766  
4  Portal hypertension   5723    K766  
         subject_id   hadm_id  seq_num icd_code  icd_version         new  \
4756321    19990427  29695607       24  T24011A           10  10_T24011A   
4756322    19996016  28015466        4   O30093           10   10_O30093   
4756323    19996783  25894657       20  H353131           10  10_H353131   
4756324    19997062  2

In [43]:
diag_d_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766
...,...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",NaN,NaN
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",NaN,NaN
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",NaN,NaN
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,NaN,NaN


In [44]:
#there are nan in the df after merge how many

diag_d_icd.isna().sum()

subject_id           0
hadm_id              0
seq_num              0
icd_code             0
icd_version          0
new                  0
long_title           0
icd9cm         2019269
icd10cm        2019269
dtype: int64

In [45]:
#which rows are null

#for icd version 38906 rows are nan out of 2766877

icd9_nan = (diag_d_icd.loc[diag_d_icd['icd_version']==9])[diag_d_icd.isnull().any(axis=1)]  

#for icd10 1980363 rows are nan.out of 1989449

icd10_nan = (diag_d_icd.loc[diag_d_icd['icd_version']==10])[diag_d_icd.isnull().any(axis=1)]  

C:\Users\upadh\AppData\Local\Temp\ipykernel_10704\121918952.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  icd9_nan = (diag_d_icd.loc[diag_d_icd['icd_version']==9])[diag_d_icd.isnull().any(axis=1)]
C:\Users\upadh\AppData\Local\Temp\ipykernel_10704\121918952.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  icd10_nan = (diag_d_icd.loc[diag_d_icd['icd_version']==10])[diag_d_icd.isnull().any(axis=1)]


In [46]:
diag_d_icd['icd10cm'] = diag_d_icd['icd10cm'].fillna(diag_d_icd['icd_code'])

In [47]:
diag_d_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766
...,...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",NaN,T24011A
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",NaN,O30093
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",NaN,H353131
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,NaN,K8036


In [48]:
diag_d_icd.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766


In [49]:
diag_d_icd["icd"]=diag_d_icd['icd10cm'].astype(str).str[:3]
diag_d_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm,icd
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766,K76
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766,K76
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766,K76
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766,K76
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766,K76
...,...,...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",NaN,T24011A,T24
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",NaN,O30093,O30
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",NaN,H353131,H35
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,NaN,K8036,K80


In [50]:
diag_d_icd['icd'].value_counts().to_frame().head(20)

,icd
E78,141010
E11,122226
Z79,120511
I25,109100
I16,98641
I50,86399
Z87,84852
K21,80446
E87,80201
F32,66252


In [51]:
diag_d_icd['subject_id'].nunique()

180640

In [52]:
topicd_patientcount = diag_d_icd.groupby('icd')['subject_id'].nunique().sort_values(ascending=False).to_frame().reset_index().rename(columns={"subject_id": "patient_count"})

In [53]:
(diag_d_icd.groupby(['subject_id','hadm_id'])['icd'].value_counts()
 .to_frame().rename(columns={'icd':'count'})
 .sort_values('count', ascending=False)).head(50)

,,,count
subject_id,hadm_id,icd,
11600538,26442681,M25,20
11487648,29594859,M25,10
19584791,26572889,E11,10
17986879,26132686,M25,10
16335156,21280952,M25,10
15810184,20678948,M25,10
10493905,23465406,Z80,9
13341971,26295765,S02,9
11611136,23040190,O99,9


In [54]:
diag_d_icd.loc[diag_d_icd['icd']=='I16']

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm,icd
441896,10000635,26134563,3,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
441897,10000764,27897940,13,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
441898,10001176,23334588,5,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
441899,10001217,24597018,8,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
441900,10001217,27703517,6,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
...,...,...,...,...,...,...,...,...,...,...
4538539,19265652,24073740,2,I169,10,10_I169,"Hypertensive crisis, unspecified",NaN,I169,I16
4538540,19476297,20894222,1,I169,10,10_I169,"Hypertensive crisis, unspecified",NaN,I169,I16
4538541,19480985,27592726,10,I169,10,10_I169,"Hypertensive crisis, unspecified",NaN,I169,I16
4538542,19550378,27016663,1,I169,10,10_I169,"Hypertensive crisis, unspecified",NaN,I169,I16


In [51]:
topicd_patientcount.iloc[10:30]
# topicd_patientcount['patient_count'].sum() #2777986

,icd,patient_count
10,I25,33458
11,N17,31523
12,F41,27382
13,I48,27075
14,D64,26750
15,Z85,25270
16,Z86,25059
17,I50,23930
18,Z68,23309
19,F17,22908


In [55]:
diag_d_icd.loc[diag_d_icd['icd']=='N17']['long_title'].value_counts()

Acute kidney failure, unspecified                                                                       47856
Acute kidney failure with lesion of tubular necrosis                                                     3179
Acute kidney failure with tubular necrosis                                                               2514
Other acute kidney failure                                                                                145
Acute kidney failure with other specified pathological lesion in kidney                                   117
Acute kidney failure with lesion of renal medullary [papillary] necrosis                                    5
Acute kidney failure with acute cortical necrosis                                                           4
Acute kidney failure with medullary necrosis                                                                1
Nephritis and nephropathy, not specified as acute or chronic, with lesion of renal cortical necrosis        1
Name: long

In [56]:
diag_d_icd.loc[diag_d_icd['subject_id']==10000032]

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm,icd
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766,K76
3486,10000032,22595853,2,78959,9,9_78959,Other ascites,78959,R188,R18
3487,10000032,22841357,2,78959,9,9_78959,Other ascites,78959,R188,R18
3488,10000032,25742920,2,78959,9,9_78959,Other ascites,78959,R188,R18
3489,10000032,29079034,5,78959,9,9_78959,Other ascites,78959,R188,R18
8432,10000032,22595853,3,5715,9,9_5715,Cirrhosis of liver without mention of alcohol,5715,K7460,K74
8433,10000032,22841357,6,5715,9,9_5715,Cirrhosis of liver without mention of alcohol,5715,K7460,K74
8434,10000032,25742920,4,5715,9,9_5715,Cirrhosis of liver without mention of alcohol,5715,K7460,K74
8435,10000032,29079034,13,5715,9,9_5715,Cirrhosis of liver without mention of alcohol,5715,K7460,K74
14231,10000032,22595853,4,07070,9,9_07070,Unspecified viral hepatitis C without hepatic ...,07070,B1920,B19


The diagnostic patients df diag has 4756326 rows and has 180640 unique subject_id.

In [57]:
diag['subject_id'].nunique() 

180640

In [58]:
admissions = (pd.read_csv('../data/hos/admissions.csv', 
                          header=0,
                          index_col=None,
                          parse_dates=['admittime', 'dischtime', 'deathtime'])
              .reset_index(drop = True))
admissions

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


In [59]:
#admissions has431231 rows with 180733 unique patients subjectID

admissions['subject_id'].nunique()
admissions['deathtime'].isnull().sum() #422633
admissions['race'].value_counts()
admissions.loc[admissions['deathtime']!= 'NaT'].count() #8598
(admissions.loc[admissions['deathtime']!= 'NaT'])['race'].value_counts()

WHITE                                        272932
BLACK/AFRICAN AMERICAN                        59959
OTHER                                         15102
UNKNOWN                                       10668
HISPANIC/LATINO - PUERTO RICAN                 8076
WHITE - OTHER EUROPEAN                         7925
HISPANIC OR LATINO                             7754
ASIAN                                          6156
ASIAN - CHINESE                                5587
WHITE - RUSSIAN                                5024
BLACK/CAPE VERDEAN                             4765
HISPANIC/LATINO - DOMINICAN                    4383
BLACK/CARIBBEAN ISLAND                         2704
BLACK/AFRICAN                                  2530
PATIENT DECLINED TO ANSWER                     1761
UNABLE TO OBTAIN                               1603
PORTUGUESE                                     1510
ASIAN - SOUTH EAST ASIAN                       1446
HISPANIC/LATINO - GUATEMALAN                   1330
ASIAN - ASIA

In [60]:
pts = (pd.read_csv('../data/hos/patients.csv', header=0, index_col=None, parse_dates=['dod'])
       .fillna(0).reset_index(drop = True))
pts

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09 00:00:00
1,10000048,F,23,2126,2008 - 2010,0
2,10000068,F,19,2160,2008 - 2010,0
3,10000084,M,72,2160,2017 - 2019,2161-02-13 00:00:00
4,10000102,F,27,2136,2008 - 2010,0
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,0
299708,19999829,F,28,2186,2008 - 2010,0
299709,19999840,M,58,2164,2008 - 2010,2164-09-17 00:00:00
299710,19999914,F,49,2158,2017 - 2019,0


In [61]:
# Get the MAXIMUM possible year that the anchor_year corresponds to
pts['min_year_group'] = pts.anchor_year_group.str.slice(start=0, stop=4).astype(int)
pts['max_year_group'] = pts.anchor_year_group.str.slice(start=-4).astype(int)

# Youngest possible age occurs by finding the diff between the max year and 2008, then subtracting from anchor_age
# This ensures you don't accidentally include someone who might've been <18 in that anchor year range during the 2008-2019 window
pts['min_age'] = pts.anchor_age.values - (pts.max_year_group.values - 2008)
# pts = pts.loc[pts.min_age >= 18]

# gets a patient's specific anchor year that corresponds to the range 2008-2010
# This year is timedelta = 0 for the end dataset
pts['base_anchor_year'] = (pts.anchor_year - (pts.min_year_group - 2008))

In [62]:
pts

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,min_year_group,max_year_group,min_age,base_anchor_year
0,10000032,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
1,10000048,F,23,2126,2008 - 2010,0,2008,2010,21,2126
2,10000068,F,19,2160,2008 - 2010,0,2008,2010,17,2160
3,10000084,M,72,2160,2017 - 2019,2161-02-13 00:00:00,2017,2019,61,2151
4,10000102,F,27,2136,2008 - 2010,0,2008,2010,25,2136
...,...,...,...,...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,0,2017,2019,35,2138
299708,19999829,F,28,2186,2008 - 2010,0,2008,2010,26,2186
299709,19999840,M,58,2164,2008 - 2010,2164-09-17 00:00:00,2008,2010,56,2164
299710,19999914,F,49,2158,2017 - 2019,0,2017,2019,38,2149


In [63]:
admissions.columns.to_list()

['subject_id',
 'hadm_id',
 'admittime',
 'dischtime',
 'deathtime',
 'admission_type',
 'admit_provider_id',
 'admission_location',
 'discharge_location',
 'insurance',
 'language',
 'marital_status',
 'race',
 'edregtime',
 'edouttime',
 'hospital_expire_flag']

In [64]:
admissions[['subject_id','language', 'marital_status','race']].drop_duplicates()

,subject_id,language,marital_status,race
0,10000032,ENGLISH,WIDOWED,WHITE
4,10000068,ENGLISH,SINGLE,WHITE
5,10000084,ENGLISH,MARRIED,WHITE
7,10000108,ENGLISH,SINGLE,WHITE
8,10000117,ENGLISH,DIVORCED,WHITE
...,...,...,...,...
431207,19999733,ENGLISH,SINGLE,WHITE
431208,19999784,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN
431226,19999828,ENGLISH,SINGLE,WHITE
431228,19999840,ENGLISH,WIDOWED,WHITE


In [66]:
pts.shape #(299712, 10)
# pts.to_csv('../data/pts.csv')

In [67]:
(pts.merge((admissions[['subject_id','language', 'marital_status','race']].drop_duplicates()),
          how='left', on=['subject_id']))

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,min_year_group,max_year_group,min_age,base_anchor_year,language,marital_status,race
0,10000032,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174,ENGLISH,WIDOWED,WHITE
1,10000048,F,23,2126,2008 - 2010,0,2008,2010,21,2126,NaN,NaN,NaN
2,10000068,F,19,2160,2008 - 2010,0,2008,2010,17,2160,ENGLISH,SINGLE,WHITE
3,10000084,M,72,2160,2017 - 2019,2161-02-13 00:00:00,2017,2019,61,2151,ENGLISH,MARRIED,WHITE
4,10000102,F,27,2136,2008 - 2010,0,2008,2010,25,2136,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
317164,19999828,F,46,2147,2017 - 2019,0,2017,2019,35,2138,ENGLISH,SINGLE,WHITE
317165,19999829,F,28,2186,2008 - 2010,0,2008,2010,26,2186,NaN,NaN,NaN
317166,19999840,M,58,2164,2008 - 2010,2164-09-17 00:00:00,2008,2010,56,2164,ENGLISH,WIDOWED,WHITE
317167,19999914,F,49,2158,2017 - 2019,0,2017,2019,38,2149,NaN,NaN,NaN


In [68]:
pts_mod = pts[['subject_id', 'gender', 'base_anchor_year', 'min_age', 'dod']]
pts_mod

,subject_id,gender,base_anchor_year,min_age,dod
0,10000032,F,2174,44,2180-09-09 00:00:00
1,10000048,F,2126,21,0
2,10000068,F,2160,17,0
3,10000084,M,2151,61,2161-02-13 00:00:00
4,10000102,F,2136,25,0
...,...,...,...,...,...
299707,19999828,F,2138,35,0
299708,19999829,F,2186,26,0
299709,19999840,M,2164,56,2164-09-17 00:00:00
299710,19999914,F,2149,38,0


In [69]:
diag.head(25)

,subject_id,hadm_id,seq_num,icd_code,icd_version,new
0,10000032,22595853,1,5723,9,9_5723
1,10000032,22595853,2,78959,9,9_78959
2,10000032,22595853,3,5715,9,9_5715
3,10000032,22595853,4,07070,9,9_07070
4,10000032,22595853,5,496,9,9_496
5,10000032,22595853,6,29680,9,9_29680
6,10000032,22595853,7,30981,9,9_30981
7,10000032,22595853,8,V1582,9,9_V1582
8,10000032,22841357,1,07071,9,9_07071
9,10000032,22841357,2,78959,9,9_78959


In [70]:
diag_d_icd_short = diag_d_icd[['subject_id', 'hadm_id','icd10cm', 'long_title', 'icd']]

In [71]:
diag_d_icd_short

,subject_id,hadm_id,icd10cm,long_title,icd
0,10000032,22595853,K766,Portal hypertension,K76
1,10000826,20032235,K766,Portal hypertension,K76
2,10000826,28289260,K766,Portal hypertension,K76
3,10005866,26158160,K766,Portal hypertension,K76
4,10008924,23676183,K766,Portal hypertension,K76
...,...,...,...,...,...
4756321,19990427,29695607,T24011A,"Burn of unspecified degree of right thigh, ini...",T24
4756322,19996016,28015466,O30093,"Twin pregnancy, unable to determine number of ...",O30
4756323,19996783,25894657,H353131,"Nonexudative age-related macular degeneration,...",H35
4756324,19997062,20096107,K8036,Calculus of bile duct with acute and chronic c...,K80


In [72]:
# diag_d_icd_short.to_csv('../data/diag_icd.csv')

In [73]:
diag_d_icd_short.isnull().sum()

subject_id    0
hadm_id       0
icd10cm       0
long_title    0
icd           0
dtype: int64

In [74]:
pts_icd_list = diag_d_icd.groupby(['subject_id', 'hadm_id'])['icd'].apply(list)

In [75]:
pts_icd_list

subject_id  hadm_id 
10000032    22595853             [K76, R18, K74, B19, J44, F31, F43, Z87]
            22841357             [R18, K74, J44, B19, D69, E87, Z21, F17]
            25742920    [R18, K74, J44, E87, Z21, F17, B18, Z99, E87, ...
            29079034    [R18, K74, J44, F31, E87, Z21, F17, Z99, E87, ...
10000068    25022803                                                [F10]
                                              ...                        
19999828    25744818    [F41, Z87, Z79, I10, E87, Y83, Y92, J44, I48, ...
            29734428    [F41, Z87, I10, N39, Y83, E11, J44, I48, Z90, ...
19999840    21033226    [I16, E78, J98, I95, J96, J69, I69, I69, G40, ...
            26071774                  [F17, I16, E78, G40, I63, I69, I69]
19999987    23865745    [N17, I25, N39, G93, E03, C85, B96, I61, R56, ...
Name: icd, Length: 430852, dtype: object

In [76]:


# len(admissions.groupby(['subject_id', 'hadm_id']))#431231

# admissions['subject_id'].nunique() #180733

diag_d_icd['subject_id'].nunique() #180640

# pts_mod['subject_id'].nunique() #299712

180640

In [77]:
admissions.merge(pts_mod, how='inner', on=['subject_id'])

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,base_anchor_year,min_age,dod
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,F,2174,44,2180-09-09 00:00:00
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0,F,2174,44,2180-09-09 00:00:00
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0,F,2174,44,2180-09-09 00:00:00
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0,F,2174,44,2180-09-09 00:00:00
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0,F,2160,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0,F,2138,35,0
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0,F,2138,35,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1,M,2164,56,2164-09-17 00:00:00
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0,M,2164,56,2164-09-17 00:00:00


In [ ]:
#Making the icddata fames o read the txt file
#using top 20 icd with respect to number of patients



In [78]:
diag_d_icd_short.head()

,subject_id,hadm_id,icd10cm,long_title,icd
0,10000032,22595853,K766,Portal hypertension,K76
1,10000826,20032235,K766,Portal hypertension,K76
2,10000826,28289260,K766,Portal hypertension,K76
3,10005866,26158160,K766,Portal hypertension,K76
4,10008924,23676183,K766,Portal hypertension,K76


In [82]:
df_e78 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='E78'].drop(columns=['icd10cm']) #141010 rows × 5 columns

In [ ]:
# df_i16 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='I16'].drop(columns=['icd10cm'])
# df_e78 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='Y92'].drop(columns=['icd10cm'])
# df_e78 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='E78'].drop(columns=['icd10cm'])
# df_e78 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='E78'].drop(columns=['icd10cm'])
# df_e78 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='E78'].drop(columns=['icd10cm'])
# df_e78 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='E78'].drop(columns=['icd10cm'])

In [79]:
diag_d_icd.loc[diag_d_icd['icd']=='I16']['long_title'].value_counts()

Unspecified essential hypertension    97361
Hypertensive urgency                    787
Hypertensive emergency                  462
Hypertensive crisis, unspecified         31
Name: long_title, dtype: int64

In [83]:
df_e78

,subject_id,hadm_id,long_title,icd
174540,10000084,23052089,"Hyperlipidemia, unspecified",E78
174541,10000084,29888819,"Hyperlipidemia, unspecified",E78
174542,10000980,20897796,"Hyperlipidemia, unspecified",E78
174543,10000980,29659838,"Hyperlipidemia, unspecified",E78
174544,10001401,21544441,"Hyperlipidemia, unspecified",E78
...,...,...,...,...
4737496,12271405,26160561,Hyperchylomicronemia,E78
4737497,13203297,21559041,Hyperchylomicronemia,E78
4737498,15518538,26699037,Hyperchylomicronemia,E78
4737499,15952360,26317400,Hyperchylomicronemia,E78


In [80]:
diag_d_icd_short

,subject_id,hadm_id,icd10cm,long_title,icd
0,10000032,22595853,K766,Portal hypertension,K76
1,10000826,20032235,K766,Portal hypertension,K76
2,10000826,28289260,K766,Portal hypertension,K76
3,10005866,26158160,K766,Portal hypertension,K76
4,10008924,23676183,K766,Portal hypertension,K76
...,...,...,...,...,...
4756321,19990427,29695607,T24011A,"Burn of unspecified degree of right thigh, ini...",T24
4756322,19996016,28015466,O30093,"Twin pregnancy, unable to determine number of ...",O30
4756323,19996783,25894657,H353131,"Nonexudative age-related macular degeneration,...",H35
4756324,19997062,20096107,K8036,Calculus of bile duct with acute and chronic c...,K80


In [84]:
# df_e78.to_csv('../data/icd/e78.csv', index=False)

In [85]:
pd.read_csv('../data/icd/e78.csv')

,subject_id,hadm_id,long_title,icd
0,10000084,23052089,"Hyperlipidemia, unspecified",E78
1,10000084,29888819,"Hyperlipidemia, unspecified",E78
2,10000980,20897796,"Hyperlipidemia, unspecified",E78
3,10000980,29659838,"Hyperlipidemia, unspecified",E78
4,10001401,21544441,"Hyperlipidemia, unspecified",E78
...,...,...,...,...
141005,12271405,26160561,Hyperchylomicronemia,E78
141006,13203297,21559041,Hyperchylomicronemia,E78
141007,15518538,26699037,Hyperchylomicronemia,E78
141008,15952360,26317400,Hyperchylomicronemia,E78


In [86]:
df_i16 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='I16'].drop(columns=['icd10cm'])

In [89]:
df_i16

,subject_id,hadm_id,long_title,icd
441896,10000635,26134563,Unspecified essential hypertension,I16
441897,10000764,27897940,Unspecified essential hypertension,I16
441898,10001176,23334588,Unspecified essential hypertension,I16
441899,10001217,24597018,Unspecified essential hypertension,I16
441900,10001217,27703517,Unspecified essential hypertension,I16
...,...,...,...,...
4538539,19265652,24073740,"Hypertensive crisis, unspecified",I16
4538540,19476297,20894222,"Hypertensive crisis, unspecified",I16
4538541,19480985,27592726,"Hypertensive crisis, unspecified",I16
4538542,19550378,27016663,"Hypertensive crisis, unspecified",I16


In [90]:
# df_i16.to_csv('../data/icd/i16.csv', index=False)

In [91]:
df_y92 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='Y92'].drop(columns=['icd10cm'])

In [92]:
df_y92

,subject_id,hadm_id,long_title,icd
337785,10000117,27988844,Sidewalk as the place of occurrence of the ext...,Y92
337786,10013324,25696131,Sidewalk as the place of occurrence of the ext...,Y92
337787,10071847,22771062,Sidewalk as the place of occurrence of the ext...,Y92
337788,10121836,20311493,Sidewalk as the place of occurrence of the ext...,Y92
337789,10132626,28839352,Sidewalk as the place of occurrence of the ext...,Y92
...,...,...,...,...
4754823,18025951,21254731,Opera house as the place of occurrence of the ...,Y92
4754897,18098009,29981080,Swimming-pool of other non-institutional resid...,Y92
4755028,18224280,28986161,Parkway as the place of occurrence of the exte...,Y92
4755074,18263764,29622097,Garden or yard of mobile home as the place of ...,Y92


In [93]:
# df_y92.to_csv('../data/icd/y92.csv', index=False)

In [94]:
df_e11 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='E11'].drop(columns=['icd10cm'])

In [95]:
df_e11

,subject_id,hadm_id,long_title,icd
400867,10000635,26134563,Diabetes mellitus without mention of complicat...,E11
400868,10000980,24947999,Diabetes mellitus without mention of complicat...,E11
400869,10000980,26913865,Diabetes mellitus without mention of complicat...,E11
400870,10000980,29654838,Diabetes mellitus without mention of complicat...,E11
400871,10001176,23334588,Diabetes mellitus without mention of complicat...,E11
...,...,...,...,...
4752332,15957274,25701023,Type 2 diabetes mellitus with moderate nonprol...,E11
4752333,19671332,25818846,Type 2 diabetes mellitus with moderate nonprol...,E11
4753450,16674634,20079966,Type 2 diabetes mellitus with severe nonprolif...,E11
4754312,17467360,20430033,Type 2 diabetes mellitus with stable prolifera...,E11


In [96]:
# df_e11.to_csv('../data/icd/e11.csv', index=False)

In [97]:
df_n17 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='N17'].drop(columns=['icd10cm'])

In [98]:
df_n17

,subject_id,hadm_id,long_title,icd
562528,10000764,27897940,"Acute kidney failure, unspecified",N17
562529,10000980,25242409,"Acute kidney failure, unspecified",N17
562530,10001338,22119639,"Acute kidney failure, unspecified",N17
562531,10001877,21320596,"Acute kidney failure, unspecified",N17
562532,10002013,21975601,"Acute kidney failure, unspecified",N17
...,...,...,...,...
4705051,14524620,29824993,Acute kidney failure with lesion of renal medu...,N17
4705052,15592344,22802310,Acute kidney failure with lesion of renal medu...,N17
4705053,19290587,22694232,Acute kidney failure with lesion of renal medu...,N17
4752348,15970222,23853401,Acute kidney failure with medullary necrosis,N17


In [99]:
# df_n17.to_csv('../data/icd/n17.csv', index=False)

In [100]:
df_i50 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='I50'].drop(columns=['icd10cm'])

In [101]:
df_i50

,subject_id,hadm_id,long_title,icd
1030063,10000980,20897796,Acute on chronic diastolic (congestive) heart ...,I50
1030064,10002131,24065018,Acute on chronic diastolic (congestive) heart ...,I50
1030065,10002430,24513842,Acute on chronic diastolic (congestive) heart ...,I50
1030066,10003502,29011269,Acute on chronic diastolic (congestive) heart ...,I50
1030067,10010471,29842315,Acute on chronic diastolic (congestive) heart ...,I50
...,...,...,...,...
4742334,12845131,21973346,High output heart failure,I50
4742335,15848257,29754182,High output heart failure,I50
4742336,16164188,21859457,High output heart failure,I50
4742337,17936643,23236826,High output heart failure,I50


In [102]:
# df_i50.to_csv('../data/icd/i50.csv', index=False)

In [103]:
df_z85 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='Z85'].drop(columns=['icd10cm'])

In [104]:
df_z85

,subject_id,hadm_id,long_title,icd
223121,10000084,23052089,Personal history of malignant neoplasm of pros...,Z85
223122,10000084,29888819,Personal history of malignant neoplasm of pros...,Z85
223123,10001919,29897682,Personal history of malignant neoplasm of pros...,Z85
223124,10004457,25559382,Personal history of malignant neoplasm of pros...,Z85
223125,10004457,28108313,Personal history of malignant neoplasm of pros...,Z85
...,...,...,...,...
4745957,18194734,26809852,Personal history of malignant neoplasm of urin...,Z85
4745958,19491293,23097422,Personal history of malignant neoplasm of urin...,Z85
4747336,13872674,22047004,Personal history of malignant neoplasm of othe...,Z85
4747337,17722345,28304595,Personal history of malignant neoplasm of othe...,Z85


In [105]:
# df_z85.to_csv('../data/icd/z85.csv', index=False)

In [106]:
df_e87 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='E87'].drop(columns=['icd10cm'])

In [107]:
df_e87

,subject_id,hadm_id,long_title,icd
81732,10000032,22841357,Hyposmolality and/or hyponatremia,E87
81733,10000032,25742920,Hyposmolality and/or hyponatremia,E87
81734,10000032,29079034,Hyposmolality and/or hyponatremia,E87
81735,10000826,20032235,Hyposmolality and/or hyponatremia,E87
81736,10000826,21086876,Hyposmolality and/or hyponatremia,E87
...,...,...,...,...
4505795,18749714,24516345,Transfusion associated circulatory overload,E87
4505796,19520579,29532191,Transfusion associated circulatory overload,E87
4505797,19852197,25125510,Transfusion associated circulatory overload,E87
4753037,16411136,28425215,Mechanical failure of instrument or apparatus ...,E87


In [108]:
# df_e87.to_csv('../data/icd/z85.csv', index=False)

In [111]:
df_f32 = diag_d_icd_short.loc[diag_d_icd_short['icd']=='F32'].drop(columns=['icd10cm'])

In [112]:
df_f32

,subject_id,hadm_id,long_title,icd
861349,10000883,29957930,"Major depressive affective disorder, single ep...",F32
861350,10015487,23914645,"Major depressive affective disorder, single ep...",F32
861351,10030046,24012309,"Major depressive affective disorder, single ep...",F32
861352,10039070,24329176,"Major depressive affective disorder, single ep...",F32
861353,10042737,21542679,"Major depressive affective disorder, single ep...",F32
...,...,...,...,...
4743960,16225551,28769307,"Major depressive disorder, single episode, mild",F32
4743961,17055995,27032096,"Major depressive disorder, single episode, mild",F32
4743962,17904392,21093788,"Major depressive disorder, single episode, mild",F32
4751092,15353057,29702976,"Major depressive affective disorder, single ep...",F32


In [114]:
df_f32.to_csv('../data/icd/f32.csv', index=False)

admissions [subject_id, hadm_id]# admit, dis, dod, race, langaa, martial, dis
pts[subject_id] #age, gender, dod
pts_admission_icd [subject, hadm_id] #icd
